Data Exploration

In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import *
from keras.optimizers import Adam
from keras.models import Sequential
from keras import Model

Loading data

In [ ]:
#Reading all the images into a numpy array
images = []
for file in os.listdir():
  if file.endswith('.jpg'):
    #if the file is an image
    image_values = cv2.imread(file)

    #resize the image into 360 x 480 pixels
    image_values = cv2.resize(image_values,(360,480))

    images.append(image_values)

#storing everything in a numpy array
images_array = np.array(images)

Finding number of observations

In [ ]:
number_of_observations = len(images_array)
number_of_observations

31

As we are working on a image dataset we do not have any columns 

Analzing how images are stored in python variables

In [ ]:
images_array

array([[[[108, 156, 228],
         [108, 157, 229],
         [109, 158, 230],
         ...,
         [ 54,  70,  77],
         [ 54,  70,  77],
         [ 54,  70,  77]],

        [[108, 156, 228],
         [108, 157, 229],
         [109, 158, 230],
         ...,
         [ 54,  70,  77],
         [ 54,  70,  77],
         [ 54,  70,  77]],

        [[108, 156, 228],
         [108, 157, 229],
         [109, 158, 230],
         ...,
         [ 54,  70,  77],
         [ 54,  70,  77],
         [ 54,  70,  77]],

        ...,

        [[ 54,  93, 162],
         [ 54,  93, 162],
         [ 55,  94, 163],
         ...,
         [ 19,  29,  32],
         [ 18,  30,  32],
         [ 18,  30,  32]],

        [[ 54,  93, 162],
         [ 54,  93, 162],
         [ 55,  94, 163],
         ...,
         [ 19,  29,  32],
         [ 18,  30,  32],
         [ 18,  30,  32]],

        [[ 54,  93, 162],
         [ 54,  93, 162],
         [ 55,  94, 163],
         ...,
         [ 19,  29,  32],
        

In [ ]:
shape = images_array[0].shape
shape

(480, 360, 3)

In [ ]:
images_array[0]

array([[[108, 156, 228],
        [108, 157, 229],
        [109, 158, 230],
        ...,
        [ 54,  70,  77],
        [ 54,  70,  77],
        [ 54,  70,  77]],

       [[108, 156, 228],
        [108, 157, 229],
        [109, 158, 230],
        ...,
        [ 54,  70,  77],
        [ 54,  70,  77],
        [ 54,  70,  77]],

       [[108, 156, 228],
        [108, 157, 229],
        [109, 158, 230],
        ...,
        [ 54,  70,  77],
        [ 54,  70,  77],
        [ 54,  70,  77]],

       ...,

       [[ 54,  93, 162],
        [ 54,  93, 162],
        [ 55,  94, 163],
        ...,
        [ 19,  29,  32],
        [ 18,  30,  32],
        [ 18,  30,  32]],

       [[ 54,  93, 162],
        [ 54,  93, 162],
        [ 55,  94, 163],
        ...,
        [ 19,  29,  32],
        [ 18,  30,  32],
        [ 18,  30,  32]],

       [[ 54,  93, 162],
        [ 54,  93, 162],
        [ 55,  94, 163],
        ...,
        [ 19,  29,  32],
        [ 18,  30,  32],
        [ 18,  30,  32]]

Here we can observe that the images are stored pxiel wise as a 2d matrix where each element in the matrix is an array with 3 values in it. Those three values are RGB values(red green blue) of the pixel 

There are no missing values

All the pixel details in the image dataset are used as features for building our dep learning model

Target column: Our model wil be creating new face images by taking inspiration from the images we feed it. We also train a discriminator which identifies whether an image is a real one or generated one

**Steps to be performed in the project**

* I will be loading the dataset into numpy arrays. 
* I will preprocess the images like fitting its resolution. 
* I will be creating a GAN model and training it with the dataset I loaded. 
* I will be training a discriminator that can filter whether an image is real or generated
* Hyper parameter tuning will be performed
* I will be creating new images using the generator.
* For evaluation purpose, I will be taking a model available on internet to predict if generated image is a human face or not




In [ ]:
def build_discriminator(img_shape):
  model = Sequential([                                                      
  Conv2D(32, kernel_size=5, strides=2, input_shape=img_shape, padding="same"),                          
  LeakyReLU(alpha=0.2),                           
  Dropout(0.25),
  Conv2D(64, kernel_size=5, strides=2, padding="same"),                          
  BatchNormalization(momentum=0.8),                           
  LeakyReLU(alpha=0.2),                           
  Dropout(0.25),
  Conv2D(128, kernel_size=5, strides=2, padding="same"),                         
  BatchNormalization(momentum=0.8),                           
  LeakyReLU(alpha=0.2),                           
  Dropout(0.25),
  Flatten(),                            
  Dense(1),                           
  Activation("sigmoid")
  ])
                                                    
  model.summary()                           
  img = Input(shape=img_shape)                           
  d_pred = model(img)                           
  return Model(inputs=img, outputs=d_pred)

In [ ]:
def build_generator(z_dimension, channels):                           
  model = Sequential([
  Dense(2 * 120 * 90, input_dim=z_dimension),                           
  LeakyReLU(alpha=0.2),                            
  Reshape((120, 90, 2)),
  UpSampling2D(),                          
  Conv2D(128, kernel_size=5, padding="same"),                           
  BatchNormalization(momentum=0.8),                           
  LeakyReLU(alpha=0.2),
  UpSampling2D(),                           
  Conv2D(64, kernel_size=5, padding="same"),                           
  BatchNormalization(momentum=0.8),                           
  LeakyReLU(alpha=0.2),
  Conv2D(channels, kernel_size=5, padding="same"),                           
  Activation("tanh"),
  ])
  model.summary()                           
  noise = Input(shape=(z_dimension,))                           
  img = model(noise)                           
  return Model(inputs=noise, outputs=img)

In [ ]:
#load real pictures:                       
x_train = images_array                                       
# model parameters                       
img_rows = shape[0]                       
img_cols = shape[1]                       
channels = 3                       
img_shape = (img_rows, img_cols, channels)                       
z_dimension = 32                       
optimizer = Adam(0.0005, 0.5)

In [ ]:
discriminator = build_discriminator(img_shape)                       
discriminator.compile(loss='binary_crossentropy',                                             
                      optimizer=optimizer, metrics=['accuracy'])                                               
generator = build_generator(z_dimension,channels)                                               
z = Input(shape=(z_dimension,))                       
img = generator(z)                       
discriminator.trainable = False                       
d_pred = discriminator(img)                       
combined = Model(z, d_pred)                       
combined.compile(loss='binary_crossentropy',optimizer=optimizer,                                        
                 metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 240, 180, 32)      2432      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 240, 180, 32)      0         
                                                                 
 dropout (Dropout)           (None, 240, 180, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 120, 90, 64)       51264     
                                                                 
 batch_normalization (BatchN  (None, 120, 90, 64)      256       
 ormalization)                                                   
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 120, 90, 64)       0         
                                                        

In [ ]:
max_epochs = 30000
batch = 64

real_imgs = np.ones((batch, 1))
fake_imgs = np.zeros((batch, 1))

for epoch in range(max_epochs):
    index = np.random.randint(0, x_train.shape[0], batch)
    images = x_train[index]
    sample_noise = np.random.normal(0, 1, (batch, z_dimension))
    gen_images = generator.predict(sample_noise)

    fake_imgs_discriminator_loss = discriminator.train_on_batch(gen_images, fake_imgs)
    real_imgs_discriminator_loss = discriminator.train_on_batch(images, real_imgs)
    
    total_discriminator_loss = np.add(real_imgs_discriminator_loss, fake_imgs_discriminator_loss)/2
    sample_noise = np.random.normal(0, 1, (batch, z_dimension))
    g_loss = combined.train_on_batch(sample_noise, real_imgs)
    
generator.save("generator.h5" )

2/2 [==============================] - 100s 48s/step
